In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data research**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")
train.head()

In [ ]:
train.shape # data dimension

In [ ]:
train['target'].value_counts() #target variable

In [ ]:
train.isna().sum()# check if there are gaps in the data

In [ ]:
train.dtypes

# **Data preparation**

In [ ]:
# Get train data without the target and ids
X = train.iloc[:, 1:-1].copy()
# Get the target
y = train.target.copy()

# Create test X, drop ids.
test_X = test.iloc[:, 1:].copy()

In [ ]:
import gc 
import matplotlib.image as img
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, r2_score as r2
from lightgbm import LGBMRegressor

In [ ]:
# Plot
import seaborn as sns
import matplotlib.pyplot as plt

# Scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Cross-Validation
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Apply a scaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test_X = scaler.transform(test_X)

# **Building a model**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
network = models.Sequential()
network.add(layers.Dense(units=16, activation="relu", input_shape=(100,)))
network.add(layers.Dense(units=16, activation="relu"))
network.add(layers.Dense(units=1, activation="sigmoid"))

In [ ]:
network.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

In [ ]:
history = network.fit(
    X_train,
    y_train,
    epochs=11,
    verbose=1,
    batch_size=100,
    validation_data=(X_test, y_test))

In [ ]:
history = network.fit(
    X,
    y,
    epochs=11,
    verbose=1,
    batch_size=100,)

In [ ]:
y_pred = network.predict(test_X)

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns=['target'])
d = {0: '0.1', 1: '1'}
y_pred_df['target'] = y_pred_df['target'].map(d).fillna(y_pred_df['target'])
y_pred_df = pd.Series(y_pred_df['target'])
y_pred_df = y_pred_df.reset_index()
y_pred_df['index']=y_pred_df['index']+600000
y_pred_df=y_pred_df.rename(columns={'index': 'id'})
y_pred_df

In [ ]:
y_pred_df.to_csv('submission.csv', index=False)